<a href="https://colab.research.google.com/github/gurovic/MLCourse/blob/main/019_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://kaggle.com/kernels/welcome?src=https://github.com/gurovic/MLCourse/blob/main/010_read.ipynb" target="_parent"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Kaggle"></a>

# **Exploratory Data Analysis (EDA): Исследовательский анализ данных**

## **Введение в EDA**
EDA — это искусство задавать правильные вопросы данным. Это первый и самый важный этап работы с данными, где мы:
- 🕵️‍♂️ **Знакомимся** с данными
- 🔍 **Выявляем** закономерности и аномалии
- 📊 **Визуализируем** ключевые характеристики
- 💡 **Формулируем** гипотезы для дальнейшего анализа

**Цели EDA:**
1. Понимание структуры данных
2. Обнаружение выбросов и ошибок
3. Выявление взаимосвязей между переменными
4. Проверка предположений для моделей

## **🟢 Базовый уровень: Статистический анализ данных**

### **1.1 Первичное знакомство с данными**

In [ ]:
import pandas as pd
import seaborn as sns

# Загрузка данных
df = sns.load_dataset('titanic')
# iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv') # if you don't want to install seaborn


# Основная информация
print("Размер данных:", df.shape)
print("\nПервые 5 строк:")
display(df.head())

print("\nТипы данных:")
display(df.dtypes)

print("\nПропуски:")
display(df.isnull().sum())

### **1.2 Описательная статистика**

In [ ]:
# Для числовых переменных
print("Описательная статистика числовых колонок:")
display(df.describe())

# Для категориальных переменных
print("\nОписательная статистика категориальных колонок:")
display(df.describe(include=['object']))

### **1.3 Простые визуализации**

In [ ]:
import matplotlib.pyplot as plt

# Гистограмма распределения
plt.figure(figsize=(10, 6))
sns.histplot(df['age'].dropna(), kde=True, bins=20)
plt.title('Распределение возраста пассажиров')
plt.xlabel('Возраст')
plt.ylabel('Частота')
plt.show()

# Boxplot для обнаружения выбросов
plt.figure(figsize=(10, 6))
sns.boxplot(x=df['fare'])
plt.title('Распределение стоимости билетов')
plt.xlabel('Стоимость билета')
plt.ylabel('Частота')
plt.show()

### **1.4 Анализ категориальных переменных**

In [ ]:
# Мозаичный график
from statsmodels.graphics.mosaicplot import mosaic
mosaic(df, ['class', 'alive'], title='Распределение таргета по классу')
plt.show()

# Тепловая карта частот
cross_tab = pd.crosstab(df['class'], df['alive'])
sns.heatmap(cross_tab, annot=True, fmt='d', cmap='Blues')
plt.title('Тепловая карта частот')
plt.show()

## **🟡 Продвинутый уровень: Анализ взаимосвязей**

### **2.1 Корреляционный анализ**

In [ ]:
# Матрица корреляций
plt.figure(figsize=(12, 8))
corr_matrix = df.corr(numeric_only=True)
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Матрица корреляций')
plt.show()

# Парные взаимосвязи
sns.pairplot(df[['age', 'fare', 'pclass']].dropna())
plt.suptitle('Парные распределения', y=1.02)
plt.show()

### **2.2 Автоматизированный EDA: pandas profiling**

In [ ]:
# Установка библиотек (раскомментировать при первом запуске)
# библиотека pandas-profiling теперь называется ydata-profiling
!pip install ydata-profiling

from ydata_profiling import ProfileReport
profile = ProfileReport(df, title='Автоматический EDA отчет Titanic')
profile.to_file('titanic_report.html')



### **2.3 Автоматизированный EDA: sweetviz**

In [ ]:
# Установка библиотек (раскомментировать при первом запуске)
!pip install numpy sweetviz --upgrade
# После выполнения этого фрагмента кода версия numpy не будет подходить для ydata_profiling: если вы захотите запустить его, нужно удалить среду выполнения и запустить все заново

import sweetviz as sv

# удаляем колонки, чтобы работал пример, из-за бага в sweetviz при работе с новой версией numpy
df = df.drop('age', axis=1)
df = df.drop('fare', axis=1)

report = sv.analyze(df)

report.show_html('sweetviz_report.html')

## **🔴 Экспертный уровень: Продвинутые техники EDA**

### **3.1 Анализ мультиколлинеарности**

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np

# Расчет VIF
X = df.select_dtypes(include=[np.number]).dropna()
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_data.sort_values('VIF', ascending=False))

# Визуализация зависимостей
sns.pairplot(X[['survived', 'pclass', 'sibsp']], kind='reg', plot_kws={'line_kws':{'color':'red'}})
plt.show()

### **3.2 Нелинейные зависимости**

In [ ]:
from sklearn.feature_selection import mutual_info_classif
y = df['survived']
mi = mutual_info_classif(X, y)
mi_series = pd.Series(mi, index=X.columns)
mi_series.sort_values(ascending=False).plot(kind='barh')
plt.title('Mutual Information')
plt.show()



### **3.3 Большие данные**

In [ ]:
# Dask для анализа
!pip install dask
from dask.dataframe import from_pandas
ddf = from_pandas(df, npartitions=4)
ddf_grouped = ddf.groupby('sex')['pclass'].mean().compute()
print(ddf_grouped)


## **📊 Чеклист по уровням**
| Уровень | Навыки |
|---------|--------|
| 🟢 | Гистограммы, boxplot, корреляционные матрицы |
| 🟡 | 3D-визуализация, автоматизированные отчеты, временные паттерны |
| 🔴 | VIF, частичные зависимости, анализ больших данных |

## **⚠️ Антипаттерны**
1. Неправильная интерпретация корреляции как причинно-следственной связи
2. Анализ без domain knowledge (например, неучет бизнес-логики)
3. Перегрузка визуализаций (более 5-7 переменных на графике)